In [2]:
# all package needed
import mysql.connector
import datetime

In [3]:
# hyperparameters and queries



# problem b. to show courses student registered in this week
query0 = ('''
select a.name as username, c.uosname as coursename, b.year, b.Semester 
from student a, transcript b, unitofstudy c 
where a.ID = b.StudId and b.UoSCode = c.UoSCode
''')
# for detect student username and password
query1 = ("select * from student;")

# for detect courses for transcript, return all transcripts
query2 = ('''
    select a.name, c.uosname as coursename, b.UoSCode as coursenumber, Grade 
    from student a, transcript b, unitofstudy c 
    where a.ID = b.StudId and b.UoSCode = c.UoSCode;
    ''')

# for getting specific detail of transcript
query3 = ("select a.Name as username,c.uosname as coursename, b.UoSCode as coursenumber, b.Year, b.Semester, d.Enrollment, d.MaxEnrollment, e.Name as instructor, b.grade from student a, transcript b, unitofstudy c, uosoffering d, faculty e where  a.ID = b.StudId and d.InstructorId = e.id and b.Year = d.Year and b.Semester = d.Semester and d.uoscode = b.uoscode and b.uoscode = c.uoscode;")

# get course id, enhanced query0
query4 = ("select a.name as username, c.uosname as coursename, c.UoSCode as coursenumber, b.year, b.Semester from student a, transcript b, unitofstudy c where a.ID = b.StudId and b.UoSCode = c.UoSCode")
# get all courses for next quarter and year
queryEnroll1 = ('''
select distinct b.UosCode, c.uosname as coursename, b.year, b.Semester
from
transcript b, unitofstudy c
where  b.UoSCode = c.UoSCode
''')
# given course quarter, year, return its pre-requisites, 
queryEnroll2 = ('''
select a.Name as username
from student a, transcript b, uosoffering c
where a.ID = b.StudId and b.UoSCode = c.UoSCode and b.grade is not null and c.Enrollment < c.MaxEnrollment
''')
# determine the course's maximum enrollment
queryEnroll3 = (
'''

select distinct e.Name as username, e.uosname as coursename, e.UoScode as coursenumber, e.Year,e.Semester from
( select distinct a.Name, c.Semester, c.Year, d.uosname, d.UoScode
from student a, transcript b, uosoffering c, unitofstudy d
where a.ID = b.StudId and c.UoSCode = d.UoSCode  and c.UoSCode not in (select UoSCode from  transcript)) e

where (Name = '%s' and year = %s and semester = '%s')
or (Name = '%s' and year = %s and semester = '%s');

''')
# Plug in transcript table with a NULL grade

# Update Enrollment in course by adding 1



In [5]:
def quarterDetermine(month):
    if(month >= 9 and month <= 12):
        quarter = "Q1";
    elif(month >= 1 and month <= 3):
        quarter = "Q2";
    elif(month >= 4 and month <= 6):
        quarter = "Q3";
    else:
        quarter = "Q4";
    return quarter

def dateReturn():
    # return year, quarter, month, day
    year = datetime.datetime.now().year
    month = datetime.datetime.now().month
    
    quarter = quarterDetermine(month)
    return year, quarter, month

def nextQuarterReturn():
    year, quarter, month = dateReturn()
    if (month + 3) // 12 == 1:
        year += 1
    quarter = quarterDetermine((month + 3)%12)
    return year, quarter, (month + 3) % 12

        
def connecter():
    cnx = mysql.connector.connect(user='root', password='hhy7814385',
                              host='127.0.0.1',
                              database='project3-nudb')
    return cnx

def executor(query, cnx):
    cursor = cnx.cursor()
    cursor.execute(query)
    return cursor

def closer(cursor, cnx):
    cnx.commit()
    cursor.close()
    cnx.close()
    
def login(cursor, username, password, success = 0):
    # Question A
    if not cursor:
        print("Wrong Detected")
        raise
    else:
        for result in cursor:
            # print(result)
            if username == result[1] and password == result[2]:
                # print('Login success.')
                student_id = result[0]
                result = drawStudentMenu(username)
                judgeFunctions(result, username = username, student_id = student_id)
                success = 1
                
            elif username in result and password not in result:
                print('Wrong Password. Try Again')

        # not founded
        if success == 0:
            print("No valid user found")
            
def drawStudentMenu(username):
    # question b
    year, quarter, month = dateReturn()
    cnx0 = connecter()
    cursor0 = executor(query0, cnx0)
    
    print("*************************************")
    print("*************************************")
    print("***NU student DB Management System***")
    print("\n")
    print("*****there should be your course*****")
    for course in cursor0:
        if course[0] == username and course[2] == year and course[3] == str(quarter):
            print(course[1])
    print("******quarter: %s, year: %s******" %(quarter, year))
    print("\n")
    print("************Student Menu*************")   
    print("*****Please provide input below.*****")
    print("********Enter 1: Transcript**********")
    print("**********Enter 2: Enroll************")
    print("*********Enter 3: Withdraw***********")
    print("*****Enter 4: Personal Details*******")    
    print("**********Enter 0: log out **********")
    result = input("please specify function you want to do:")
    
    cnx0.commit()
    cursor0.close()
    cnx0.close()
    return result


def judgeFunctions(Input, username, student_id):
    # Interact session
    Input = int(Input)
    if Input == 1:
        print("This should be function 1: Transcript")
        returnTrans(username, student_id)
        
    elif Input == 2:
        print("This should be function 2: Enroll")
        Enrolled(username, student_id)
        
          
    elif Input == 3:
        print("This should be function 3: Withdraw")
        Withdraw(student_id)  
    elif Input == 4:
        print("This should be function 4: Personal Details")
        personalDetails(username, student_id)
    elif Input == 0:
        print("Logout Success")
        pass
    else:
        print("Error Detected.")
        raise
        
def transIteration(results, username, query = query3):
    # question b
    outputted = 0
    nextCourseNum = input("Input course number to get class detail. Enter 0 to get back.")
    if nextCourseNum == '0':
        pass
    
    else:  
        cnx2 = connecter()
        cursor2 = cnx2.cursor()
        cursor2.execute(query)

        for result in cursor2:
            if result[2] == nextCourseNum and result[0] == username:
                print(result)
                outputted = 1
        cnx2.commit()
        cursor2.close()
        cnx2.close()
        
    if outputted == 0:
        print("No courses Founded.")
    Continue = input("If finished, Enter 0, else Enter 1")
    return Continue

def returnTrans(username, student_id, query2 = query2):
    print(username)
    # question C
    cnx1 = connecter()
    cursor1 = cnx1.cursor()
    cursor1.execute(query2)
    results = []
    for result in cursor1:
        # 'Linda Smith', 'Algorithms and Complexity', 'CR'
        # print(result)
        if username in result:
            # print(result[username])
            # cursor.execute(quer
            results.append(result)
            print(result)

    cnx1.commit()
    cursor1.close()
    cnx1.close()

    Continue = transIteration(results, username)
    while Continue != '0':
        Continue = transIteration(results, username)
    further = drawStudentMenu(username)
    judgeFunctions(further, username=username, student_id=student_id) 
    

def Enrolled(username, student_id):
    # Question d
    
    currentYear, currentQuarter, currentMonth = dateReturn()
    preYear, preQuarter, preMonth = nextQuarterReturn()
    cnx4 = connecter()
    
    # get course list:
    cursor4 = cnx4.cursor()
    paras = (username, currentYear, currentQuarter, username, preYear, preQuarter)
    cursor4.execute(queryEnroll3%paras)
    # courseList = []
    print(paras)
    print('we have following courses:')
    for result in cursor4:
        print(result)

    cnx4.commit()
    cursor4.close()
    cnx4.close()

    # see if the course exceed enroll limitation
    registerRequest = input('please input the course Code you want to add(8 Digit):')
    quarter = input('Please input the quarter you want to register(like Q1)')
    year = int(input("Please input the year you want to "))
    
    
    # plug in
    cnx11 = connecter()
    cursor11 = cnx11.cursor()
    # student_ID INT, in class_ID CHAR(8), in year_num int, in semester_num char(2)
    paras = (int(student_id), registerRequest, int(year), quarter)    
    cursor11.callproc('Enroll1', paras)

    cnx11.commit()
    
    cursor11.close()
    cnx11.close()
    print('register success for course: %s'%registerRequest)
    further = drawStudentMenu(username)
    judgeFunctions(further, username=username, student_id=student_id) 
    
def Withdraw(student_id):
    # question e
    year, quarter, month = dateReturn()
    # print all courses
    print('You have following courses:')
    cnx6 = connecter()
    cursor6 = executor(query4, cnx6)
    
    for course in cursor6:
        if course[0] == username and course[3] == year and course[4] == str(quarter):
            print(course[1], course[2])
            
    cnx6.commit()
    cursor6.close()
    cnx6.close()
    
    class_id = input('please input your clss name below(eight CHARS)')
    cnx5 = connecter()
    cursor5 = cnx5.cursor()
    paras = [class_id, student_id]
    #withdraw_course varchar(20),in student_id varchar(20))
    cursor5.callproc('withdraww', paras)
    cnx5.commit()
    cursor5.close()
    cnx5.close()    
    print("Withdraw Success! Now you have following courses:")
    cnx6 = connecter()
    cursor6 = executor(query4, cnx6)
    
    for course in cursor6:
        if course[0] == username and course[3] == year and course[4] == str(quarter):
            print(course[1], course[2])
    cnx6.commit()
    cursor6.close()
    cnx6.close()
    back = input("press 1 to get another deletion, press 0 to get back to main menu")
    if back == '1':
        Withdraw(student_id)
    elif back == '0':
        further = drawStudentMenu(username)
        judgeFunctions(further, username=username, student_id=student_id) 

def personalDetails(username, student_id):
    # question f
    cnx7 = connecter()
    cursor7 = executor(query1, cnx7)
    
    for result in cursor7:
        print(result)
        if username in result:
            
            changes = input('Do you want to make any changes? 1 for yes and 0 for no')
            if changes == '1':
                details = input('which part do you want to change? 1 for password and 2 for address')
                if details == '1':
                    passwordNew = input('Input your password below')
                    query = ("UPDATE v_student SET Password = '%s' where Name = '%s'"%(passwordNew, username))
                    print(query)

                if details == '2':
                    addressNew = input('Input your address below')
                    query = ("UPDATE v_student SET Address = '%s' where Name = '%s'"%(addressNew, username))
                
                # update safe version
                cnx_sub = connecter()
                query9 = ('SET SQL_SAFE_UPDATES=0;')
                
                cursor_sub = executor(query9, cnx_sub)

                cnx_sub.commit()
                cursor_sub.close()
                cnx_sub.close()
                
                # update password or details
                cnx8 = connecter()
                cursor8 = executor(query, cnx8)

                cnx8.commit()
                cursor8.close()
                cnx8.close()
                print('changes success')
                further = drawStudentMenu(username)
                judgeFunctions(further, username=username, student_id=student_id) 
            else:
                further = drawStudentMenu(username)
                judgeFunctions(further, username=username, student_id=student_id) 
    
    cnx7.commit()
    cursor7.close()
    cnx7.close()



In [7]:
if __name__ == '__main__':
    cnx = connecter()
    cursor = executor(query1, cnx)
    
    username = input("input username:")
    password = input("input password:")
    
    login(cursor, username, password)
    
    closer(cursor, cnx)
    

input username:Linda Smith
input password:lunch
*************************************
*************************************
***NU student DB Management System***


*****there should be your course*****
Enterprise Scale Software Architecture
Introduction to IT
Introduction to Programming
Database Systems I
Object Oriented Design
Management of IT Projects and Systems
******quarter: Q1, year: 2018******


************Student Menu*************
*****Please provide input below.*****
********Enter 1: Transcript**********
**********Enter 2: Enroll************
*********Enter 3: Withdraw***********
*****Enter 4: Personal Details*******
**********Enter 0: log out **********
please specify function you want to do:2
This should be function 2: Enroll
('Linda Smith', 2018, 'Q1', 'Linda Smith', 2019, 'Q2')
we have following courses:
('Linda Smith', 'Computational Geometry', 'COMP5045', 2018, 'Q1')
('Linda Smith', 'Graphics and Multimedia', 'COMP3419', 2018, 'Q1')
('Linda Smith', 'Operating Systems and